This colab details the privacy accounting of a few models trained with Matrix-Factorization-Differentially-Private-Follow-The-Regularized-Leader (MF-DP-FTRL) by [(Amplified) Banded Matrix Factorization: A unified approach to private training](https://arxiv.org/abs/2306.08153) in cross-device federated learning.

# Preliminaries

Install and import packages including tensorflow and dp_accounting. The dp_accounting library will only be used for the [zCDP](https://arxiv.org/abs/1605.02065) to $(\epsilon, \delta)-$DP conversion.

In [ ]:
import functools
import sys

import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

! pip install dp-accounting==0.4.3
import dp_accounting

## Downloads matrices $B, C$ for MF-DP-FTRL.

Recall that we use the inverse of matrix $C$ to generate round-correlated noise, and for training round $i$, we have
$\hat{x}_i \gets x_i + \zeta [C^{-1}z]_{[i,:]}$, where $x_i$ is a $d-$dimentional vector represents the aggregated signal of client updates; $\zeta$ is a scalar of the $L_2$ clipping norm for each client update to control sensitivity; $z$ is the $(n,d)$-dimentional matrix of Gaussian noise with zero mean and standard deviation $\sigma$, and $n$ is the total number of rounds. The $i$th row of the correlated noise $[C^{-1}z]_{[i,:]}$ is applied to privatirized the update $x_i$ in the $i$th round. We then use the privatirized update $\hat{x}_i$ to update the global model.

In addition to providing the $C$ matrix in advance, the users would also need to specify hyperparameters $\zeta$ for $L_2$ clipping norm of client updates and $\sigma$ for noise multiplier w.r.t the $L_2$ clip norm, when run the MF-DP-FTRL algorithm for training, and $\sigma$ has significant impact on the privacy guarantees in accounting.

$B, C$ matrices are generated by optimizing an objective of expected total squared error given the contraint that $A=BC$ where $A$ is a $(n, n)-$ dimentional lower trianglar with $A_{[i,j]}= 1, \forall i\leq j$, $A_{[i,j]}= 0, \forall i > j$ and $C$ is a $(n, n)-$ dimentional matrix of both lower trianglar and $\hat{b}$-banded with $C_{[i,j]}= 0, \forall i > j$, and $C_{[i,j]}=0, \forall |i-j| \geq \hat{b}$. See [(Amplified) Banded Matrix Factorization: A unified approach to private training](https://arxiv.org/abs/2306.08153) for more details.



In [ ]:
data_location = 'https://github.com/google-research/federated/raw/master/mf_dpftrl_matrices/{}'
b1_url = data_location.format('bands400_rounds2000/b_matrix_tensor_pb')
c1_url = data_location.format('bands400_rounds2000/c_matrix_tensor_pb')
lr1_url = data_location.format('bands400_rounds2000/lr_vector_tensor_pb')

b2_url = data_location.format('bands1000_rounds2000/b_matrix_tensor_pb')
c2_url = data_location.format('bands1000_rounds2000/c_matrix_tensor_pb')
print(f'Data location:\n {b1_url}\n {c1_url}\n {lr1_url}\n {b2_url}\n {c2_url}')

Data location:
 https://github.com/google-research/federated/raw/master/mf_dpftrl_matrices/bands400_rounds2000/b_matrix_tensor_pb
 https://github.com/google-research/federated/raw/master/mf_dpftrl_matrices/bands400_rounds2000/c_matrix_tensor_pb
 https://github.com/google-research/federated/raw/master/mf_dpftrl_matrices/bands400_rounds2000/lr_vector_tensor_pb
 https://github.com/google-research/federated/raw/master/mf_dpftrl_matrices/bands1000_rounds2000/b_matrix_tensor_pb
 https://github.com/google-research/federated/raw/master/mf_dpftrl_matrices/bands1000_rounds2000/c_matrix_tensor_pb


In [ ]:
b1_file = tf.keras.utils.get_file(fname='b1', origin=b1_url)
c1_file = tf.keras.utils.get_file(fname='c1', origin=c1_url)
lr1_file = tf.keras.utils.get_file(fname='lr1', origin=lr1_url)

b2_file = tf.keras.utils.get_file(fname='b2', origin=b2_url)
c2_file = tf.keras.utils.get_file(fname='c2', origin=c2_url)
print(
    f'Local data location:\n {b1_file}\n {c1_file}\n {lr1_file}\n {b2_file}\n'
    f' {c2_file}'
)

In [ ]:
def load_matrix(file_path: str) -> tf.Tensor:
  return tf.io.parse_tensor(tf.io.read_file(file_path), tf.float64)


b1_matrix = load_matrix(b1_file)
c1_matrix = load_matrix(c1_file)
lr1_vector = load_matrix(lr1_file)

b2_matrix = load_matrix(b2_file)
c2_matrix = load_matrix(c2_file)

## Verify and visualize the matrices

We have loaded two set of matrices $(B1, C1)$ with a learning rate decaying vector $LR1$ for total 2000 training rounds with 400 bands; and $(B2, C2)$ for total 2000 trainig rounds with 1000 bands.

In [ ]:
def verify_and_visualize(
    b_matrix: tf.Tensor,
    c_matrix: tf.Tensor,
    bands: int,
    lr_vector: tf.Tensor | None = None,
) -> None:

  if not verify_lower_trianglar_and_banded(c_matrix.numpy(), bands):
    raise ValueError(f'C matrix has to be lower trianglar and {bands} bands.')

  a_matrix = b_matrix @ c_matrix
  if lr_vector is not None:
    a_matrix = tf.math.divide_no_nan(
        a_matrix, tf.broadcast_to(lr_vector, tf.shape(a_matrix))
    )
  if not verify_lower_trianglar_ones(a_matrix.numpy()):
    raise ValueError(f'A matrix has to be lower trianglar with 1s.')

  c_inv = np.linalg.inv(c_matrix.numpy())

  fig, axs = plt.subplots(nrows=1, ncols=4, figsize=(16, 5))
  stats = statistics(b_matrix.numpy())
  print('B matrix value (mean, std, min, max):', stats)
  b_im = axs[0].matshow(
      b_matrix.numpy(),
      vmin=max(stats[2], stats[0] - stats[1] * 2),
      vmax=min(stats[3], stats[0] + stats[1] * 2),
  )
  axs[0].title.set_text('B matrix')
  fig.colorbar(b_im, shrink=0.5)

  stats = statistics(c_matrix.numpy())
  print('C matrix value (mean, std, min, max):', stats)
  c_im = axs[1].matshow(
      c_matrix.numpy(),
      vmin=max(stats[2], stats[0] - stats[1] * 2),
      vmax=min(stats[3], stats[0] + stats[1] * 2),
  )
  axs[1].title.set_text('C matrix')
  fig.colorbar(c_im, shrink=0.5)

  stats = statistics(c_inv)
  print('C inverse matrix value (mean, std, min, max):', stats)
  c_inv_im = axs[2].matshow(
      c_inv,
      vmin=max(stats[2], stats[0] - stats[1] * 2),
      vmax=min(stats[3], stats[0] + stats[1] * 2),
  )
  axs[2].title.set_text('C inverse matrix')
  fig.colorbar(c_inv_im, shrink=0.5)

  a_im = axs[3].matshow(a_matrix.numpy(), vmin=0, vmax=1)
  axs[3].title.set_text('A matrix')
  fig.colorbar(a_im, shrink=0.5)
  plt.show()


def verify_lower_trianglar_and_banded(mat: np.ndarray, bands: int) -> bool:
  if not np.allclose(mat, np.tril(mat)):
    return False
  for i in range(mat.shape[0]):
    for j in range(i + 1):
      if (i - j) >= bands and mat[i, j] != 0:
        return False
  return True


def verify_lower_trianglar_ones(mat: np.ndarray) -> bool:
  return np.allclose(mat, np.tril(np.ones_like(mat)))


def statistics(mat: np.ndarray) -> tuple[float, float, float, float]:
  mean = np.mean(mat)
  std = np.std(mat)
  return mean, std, np.min(mat), np.max(mat)

In [ ]:
print('Loaded matrices for 400 bands and 2000 rounds.')
verify_and_visualize(b1_matrix, c1_matrix, bands=400, lr_vector=lr1_vector)
print('Loaded matrices for 1000 bands and 2000 rounds.')
verify_and_visualize(b2_matrix, c2_matrix, bands=1000)

# Privacy accounting for MF-DP-FTRL

Now we show the privacy accounting for DP models trained with MF-DP-FTRL. We consider the production setting in [Federated Learning of Gboard Language Models with Differential Privacy](https://arxiv.org/abs/2305.18465) with multiple participation and no amplification by sampling.

Recall the DP process of $\hat{x} \gets x + \zeta C^{-1}z$, which is equivalent to $A\hat{x} \gets B(Cx + \zeta z)$ with $A, B$ matrices for post-processing. Note that the objective in [(Amplified) Banded Matrix Factorization: A unified approach to private training](https://arxiv.org/abs/2306.08153) for facorizing $B,C$ is independent to the data $x$.

The key idea of privacy accounting is to look at the DP release of $Cx + \zeta z$, and compute the sensitivity of $Cx$ to calibrate with the Gaussian noise $z$ of zero mean and $\sigma$ standard deviation. We use the neighboring dataset definition in [Practical and Private (Deep) Learning without Sampling or Shuffling
](https://arxiv.org/abs/2103.00039): the two neighboring dataset $(x, x')$ differs by only zeroing out the updates of one participating device. And we compute the worst-case sensitivity of $\| \max_{x, x'} C (x-x')\|_F$. One important property we leverage is: the adaptive DP guarantee of streaming matrix mechanisms (i.e., lower triangular matrices) can be directly inferred from their non-adaptive DP guarantees, as discussed in [Improved Differential Privacy for SGD via Optimal Private Linear Operators on Adaptive Streams](https://arxiv.org/abs/2202.08312).

If each client device will only participate once, then $(x, x')$ only have difference in one row, and the sensitivity can be computed by $\max_{x, x'} \| C (x-x')\|_F = \zeta \max_{j} \| C_{[:,j]} \|$ as $\zeta$ is the $L_2$ clipping norm for each client update.

In cross-device federated learning, a device can participate multiple times. However, we also use a policy to restrict the frequency of client device participation. Specifically, a timer on each client is locally checked so that the client is only eligible for training in this round after a specified period of time since the client's previous training round. The policy results in two important parameters for privacy accounting
 -  *max participation*: the maximum number of times each client may participate in the total number of training rounds
 - *min separation b*: the minimum number of rounds between two participation of each client. Note that we use the definition in [Practical and Private (Deep) Learning without Sampling or Shuffling
](https://arxiv.org/abs/2103.00039), which is off by one compared to [(Amplified) Banded Matrix Factorization: A unified approach to private training](https://arxiv.org/abs/2306.08153). For a client participate in round $r_1$ and round $r_2$, the min separation $b=r_2-r_1-1$.

When $C$ is banded with $\hat b \leq b+1$, the computation of sensitivity for multiple participation of  is significantly simplified as $\max_{x, x'} \| C (x-x')\|_F  = \zeta \max_{\Pi} \sqrt{\sum_{j \in \Pi} \| C_{[:,j]} \|^2}$, where $\Pi$ represents the participation pattern of each client. The sensitivity can be efficiently computed by dynamic programming.



In [ ]:
def square_sensitivity_c_banded(
    c_matrix: np.ndarray, min_separation: int, max_participation: int
) -> float:
  """Computes sensitivity when C is banded and min separation is large.

  This is a recursive version of Algorithm 5 in
  [(Amplified) Banded Matrix Factorization: A unified approach to private
  training](https://arxiv.org/abs/2306.08153).

  Arguments:
    c_matrix: The C matrix used in the MF-DP-FTRL algorithm.
    min_separation: Minimum separation between two adjacent participation of
      each client. If a client participates in round $r_1$ and round $r_2$, the
      min separation is r_2-r_1-1.
    max_participations: The possible maximum number of participations of each
      client.

  Returns:
    The squared sensitivity.
  """
  total_rounds = c_matrix.shape[1]

  def _check_possible_participation(num_participation: int, start: int) -> bool:
    """Checks if the number of participation is possible.

    The participatoins are in rounds [start, total_rounds) with min_separation.
    """
    return start + (min_separation + 1) * (num_participation - 1) < total_rounds

  while not _check_possible_participation(max_participation, start=0):
    print(
        f'Warning: the max participation {max_participation} is large for '
        f'{total_rounds} rounds with min separation {min_separation}. '
        f'Reducing the max participation to {max_participation-1}.'
    )
    max_participation -= 1

  c_col_sq = np.sum(c_matrix**2, axis=0)

  @functools.lru_cache(maxsize=None)
  def _sensitivity_c_recur(num_participation: int, start: int) -> float:
    """Recursively computes the squared sensitivity for num_participation.

    Considers rounds [start, total_rounds), i.e., c_col_sq[start:].
    """
    if num_participation == 0:
      return 0.0
    elif not _check_possible_participation(num_participation, start):
      return -np.inf
    else:
      return max(
          c_col_sq[start]
          + _sensitivity_c_recur(
              num_participation - 1, start + min_separation + 1
          ),
          _sensitivity_c_recur(num_participation, start + 1),
      )

  return _sensitivity_c_recur(max_participation, 0)


def privacy_accounting(
    c_matrix: np.ndarray,
    min_seperation: int,
    max_participation: int,
    l2_clip_norm_noise_multiplier: float,
    target_delta: float | None = None,
) -> tuple[float, float | None]:
  """Computes DP guarantees for MF-DP-FTRL.

  The total sensitivity comes from the l2_clip_norm and the matrix factorization
  mechanism, and takes into account the C matrix. The effective
  `total_noise_multiplier` w.r.t. the total sensitivity is used for privacy
  accounting of Gaussian mechanism. Both Zero-Concentrated Differential Privacy
  (zCDP) and (epsilon, delta)-DP guarantees can be computed.

  Arguments:
    c_matrix: The C matrix used in the MF-DP-FTRL algorithm.
    min_separation: Minimum separation between two adjacent participation of
      each client. If a client participates in round $r_1$ and round $r_2$, the
      min separation is r_2-r_1-1.
    max_participations: The possible maximum number of participations of each
      client.
    l2_clip_norm_noise_multiplier: The noise multiplier w.r.t. the L2 clipping
      norm used when running the MF-DP-FTRL algorithm for model training.
    target_delta: The target delta used for (epsilon, delta)-DP. If None, no
      (epsilon, delta)-DP guarantee is computed, and only zCDP is returned.

  Returns:
    A tuple of values for two kinds of DP guarantees
      (zCDP, epsilon with `target_delta`)
      or a scalar of zCDP if `target_delta` is None.
  """
  sens_sq = square_sensitivity_c_banded(
      c_matrix, min_seperation, max_participation
  )
  total_noise_multiplier = l2_clip_norm_noise_multiplier / sens_sq**0.5

  zcdp = 1 / (2 * total_noise_multiplier**2)

  if target_delta is not None:
    # PLD accounting from https://github.com/google/differential-privacy is used
    # for (epsilon, delta)-DP.
    accountant = dp_accounting.pld.PLDAccountant()
    accountant.compose(dp_accounting.GaussianDpEvent(total_noise_multiplier), 1)
    eps = accountant.get_epsilon(target_delta)
    return zcdp, eps
  else:
    return zcdp

## Gboad es-ES NWP LM

The Gboard Spanish model in Spain is trained with the loaded matrix $C_1$ of 2000 rounds and 400 bounds, noise multiplier 1.411; the min separation is 484, and the max participation is 3.

In [ ]:
sys.setrecursionlimit(2000 * 3)  # total_rounds * max_participation
zcdp, eps = privacy_accounting(
    c1_matrix,
    min_seperation=484,
    max_participation=3,
    l2_clip_norm_noise_multiplier=1.411,
    target_delta=1e-10
)
print(f'DP guarantees: zcdp = {zcdp}, (eps, delta) = ({eps}, 1e-10)')

DP guarantees: zcdp = 0.1506840301548885, (eps, delta) = (3.4240074094281336, 1e-10)


## Gboad pt-BR NWP LM

 The Portuguese model in Brazil is trained with the loaded matrix of $C_2$ of 2000 rounds and 1000 bands, noise multiplier of 8.35; the min separation is 1428 and the max participation is 2.

In [ ]:
sys.setrecursionlimit(2000 * 2)  # total_rounds * max_participation
zcdp, eps = privacy_accounting(
    c2_matrix,
    min_seperation=1428,
    max_participation=2,
    l2_clip_norm_noise_multiplier=8.35,
    target_delta=1e-10
)
print(f'DP guarantees: zcdp = {zcdp}, (eps, delta) = ({eps}, 1e-10)')

DP guarantees: zcdp = 0.014342572340349278, (eps, delta) = (0.9935500950539097, 1e-10)


## Gboad es-US NWP LM

The Spanish model in Latin America is trained with the loaded matrix of $C_2$ of 2000 rounds and 1000 bands, noise multiplier of 8.35; the min separation is 1132 and the max participation is 2.

In [ ]:
sys.setrecursionlimit(2000 * 2)  # total_rounds * max_participation
zcdp, eps = privacy_accounting(
    c2_matrix,
    min_seperation=1132,
    max_participation=2,
    l2_clip_norm_noise_multiplier=8.35,
    target_delta=1e-10
)
print(f'DP guarantees: zcdp = {zcdp}, (eps, delta) = ({eps}, 1e-10)')

DP guarantees: zcdp = 0.014342572340349278, (eps, delta) = (0.9935500950539097, 1e-10)
